In [1]:
# import
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.cm as cm 
import seaborn as sns
from scipy import signal
from importlib import reload

#import winsound as sd
import pickle
import sys
import os

from Tools.tools import *
from Plasticity.plasticity import *
from Neuron.neuron import *
from Model.model import *
from Analysis.analysis import *
import Operate_model_v3

In [2]:
# load
cd = os.getcwd()
path = os.path.dirname(cd)
    
with open(path+'//Data_final.pkl', 'rb') as file:
    # 453개의 data에 해당하는 input vector, DG vector, DG vector size, 
    Filt_idata = pickle.load(file)
    Filt_gdata = pickle.load(file)
    Filt_glen = pickle.load(file)
    F_DataLen = pickle.load(file)
    
    # CA3 vector index, CA3i vector index
    Af_py = pickle.load(file)
    Af_in = pickle.load(file)
    
    # 뭐였더라
    In_py_list = pickle.load(file)
    In_in_list = pickle.load(file)

In [4]:
class config:
    def __init__(self, pc):
        # The number of neurons
        self.N_I = 16
        self.N_O = self.N_I

        self.N_DG = 800
        self.N_DGH = self.N_I
        self.N_DGM = 100 
        self.N_DGB = 400 

        self.N_CA3 = pc[3] 
        self.N_CA1 = 200
        self.N_CA3i = pc[4] 
        
        # Weights of each connection
        # From EC
        self.w_EI = 3
        self.w_ppDG = 2 
        self.w_ppCA3 = 1 
        self.w_ppCA1 = 0.5 

        # DG
        self.w_IH = 3 
        self.w_IB = 4 
        self.w_IM = 3 
        self.w_HM = 4 
        self.w_MB = 4 
        self.w_BDG = 3 

        # CA3
        self.w_mf = 2 
        self.w_mfi = 2 
        self.w_Rc = 1

        self.w_CA33i = 1
        self.w_3iCA3 = 2 
        self.w_3i3i = 0.5 
        self.w_NoCA3 = 0.5

        # CA1
        self.w_Sc = 0.15
        self.w_CA1O = 2 

        # The number of connections 
        # From EC
        self.c_ppDG = 1
        self.c_ppCA3 = 8
        self.c_ppCA1 = 3

        # DG
        self.c_IH = 1
        self.c_IB = self.N_I
        self.c_IM = self.N_I*2//3
        self.c_HM = 1
        self.c_MB = self.N_I
        self.c_BDG = 1

        # CA3
        self.c_mf = pc[0]
        self.c_mfi = pc[1]
        self.c_Rc = self.N_CA3//4

        self.c_CA33i = self.N_CA3i//4
        self.c_3iCA3 = pc[2]
        self.c_3i3i = pc[5]
        self.c_NoCA3 = self.N_CA3
        self.c_Sc =  self.N_CA3//2

        # Parameters for delay
        self.d_I = 10
        self.d_O = 15
        self.d_DG = 3
        self.d_CA3 = 5
        self.d_CA3i = 1

In [5]:
# init
pc = [30,3,600,2400,120,20]
already_learned = False
learned_weights = [0,0,0,0]
params = config(pc)

In [10]:
# Neuron indices want to see the specific values
WTS_I = []
WTS_DGH = []
WTS_DGM = []
WTS_DGB = []
WTS_DG = []
WTS_CA3 = []
WTS_CA3i = []
WTS_CA1 = []
WTS = [WTS_I, WTS_DGH, WTS_DGM, WTS_DGB, WTS_DG, WTS_CA3, WTS_CA3i, WTS_CA1]

In [16]:
# Bias 조사
with open('Data_BiasedInputs\Cue_and_Target_Bias_Set3.pkl', 'rb') as file:
    Target_Data = pickle.load(file)
    Cue_Data = pickle.load(file)
    
learned_path = path+'//module//Weights_Bias_v2'

In [12]:
a

NameError: name 'a' is not defined

In [18]:
with open(learned_path+'//Learned weights-bias_v41-0.pkl', 'rb') as file:
    q_ppCA3 = pickle.load(file)   
    q_Rc = pickle.load(file)
    q_CA33i = pickle.load(file)
    q_Sc = pickle.load(file)

In [ ]:
# Learning
for Trial in range(25):
    Trial = Trial
    Tar = (Trial // 5) + 1
    Try = Trial % 5
    
    # initiation
    En_win = 120
    Re_win = 120
    
    Num_tar = len(Target_Data[Trial])
    Num_cue = 1
    Repeat = 1
    
    EtoI_learning = True
    already_learned = False
    learned_weights = [0,0,0,0]
    Direct_pp = True
    
    Target = Target_Data[Trial]
    Cue = np.zeros(params.N_I)

    network, Fail, If_fail, Success, Winfin, Voltage, FT, CA3_R_list, Out_R_list = Operate_model.run_model(
        En_win, Re_win, 
        Num_tar, Num_cue, Repeat, Target, Cue, 
        WTS, params, 
        already_learned, learned_weights, Direct_pp,
        Filt_idata, Filt_gdata, Af_py, Af_in)
    
    if not already_learned:
        with open('Learned weights-bias_v4%d-%d.pkl' % (Tar, Try), 'wb') as file:
            pickle.dump(network.q_ppCA3, file)
            pickle.dump(network.q_Rc, file)
            pickle.dump(network.q_CA33i, file)
            pickle.dump(network.q_Sc, file)

In [ ]:
# Retrieval
for Trial in range(5*5):
    Tar = (Trial // 5) + 1
    Try = Trial % 5
    
    # initiation
    En_win = 120
    Re_win = 120
    
    # 현 trial에서 cue의 개수
    Num_tar = len(Target_Data[Trial])
    Num_cue = 1
    Repeat = 100
    
    Phase_num = Num_cue*Repeat
    
    # target : 2x25 / P_Data : 16x25
    Target = Target_Data[Trial]
    Cue = Cue_Data[:,Trial]

    EtoI_learning = True
    already_learned = True
    Direct_pp = True
      
    if already_learned:
        with open(learned_path+'//Learned weights-bias_v2%d-%d.pkl' % (Tar, Try), 'rb') as file:
            q_ppCA3 = pickle.load(file)   
            q_Rc = pickle.load(file)
            q_CA33i = pickle.load(file)
            q_Sc = pickle.load(file)

        if not EtoI_learning:
            for n in range(params.N_CA3):
                for m in range(params.N_CA3i):
                    if q_CA33i[n,m] !=0:
                        q_CA33i[n,m] = 0.5
        learned_weights = [q_ppCA3, q_Rc, q_CA33i, q_Sc]
    else:
        learned_weights = [0,0,0,0]

    network, Fail, If_fail, Success, Winfin, CA3_R_list, CA1_R_list = Operate_model_v3.run_model(
        En_win, Re_win, 
        Num_tar, Num_cue, Repeat, Target, Cue, 
        WTS, params, 
        already_learned, learned_weights, Direct_pp,
        Filt_idata, Filt_gdata, Af_py, Af_in)
    
    with open('Results-biasM_v2%d-%d.pkl' % (Tar, Try), 'wb') as file:
        pickle.dump(CA3_R_list, file)
        pickle.dump(CA1_R_list, file)